# Calculating the credible region of the median with pymc3

Recently I had the need to compare several recommendation system algorithms by scoring how well they could rank a list of hold-out data. All performed pretty well with most of the true positives being ranked in the top ten, however the distributions of rankings were highly skewed with long tails consisting of a few poorly-ranked positives.

Observe:

![title](ranks_dist.png)


In this project, we care more about how many positives are ranked in the top ~20, and outside of this it doesn't really matter _how_ bad the ranking is, since with limited budget we are unlikely to look at predictions outside the top 20. So for scoring the algorithms, the `mean` ranking is less useful since it is highly skewed by long tails, and any change within the long tail can change the mean a lot without affecting the top 10-20 ranks at all. The `median` more accurately reflects the ranking of the majority of the ligands. 



In [ ]:
from scipy.stats import expon
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from seaborn import kdeplot
from tqdm import tqdm_notebook

In [ ]:
from scipy.stats import expon

ranks = expon(1,30).rvs(2500)

plt.hist(ranks, alpha=0.3, bins=100)
print(ranks.mean(), np.median(ranks))
plt.axvline(np.mean(ranks), label='mean')

plt.axvline(np.median(ranks), label='median')

In [ ]:
values = list()
for binsize in np.arange(50,5000,50):
    x = np.linspace(0,40,binsize)
    bap = np.abs(x-ranks[:,np.newaxis])**2
    bapsum = np.sum(bap,axis=0)

    minimum_index = np.argmin(bapsum)
    values.append(x[minimum_index])
    print(x[minimum_index])

In [ ]:
values = list()
for binsize in np.arange(50,5000,50):
    x = np.linspace(0,40,binsize)
    bap = np.abs(x-ranks[:,np.newaxis])
    bapsum = np.sum(bap,axis=0)

    minimum_index = np.argmin(bapsum)
    values.append(x[minimum_index])
    print(x[minimum_index])
    
plt.plot(values)
plt.axhline(np.median(ranks), c='k')

In [ ]:
from scipy.optimize import minimize

history = []
def callback(x):
    history.append(x)
    #fobj = l1_loss(x)
    #history.append(fobj)





In [ ]:
from scipy.optimize import fmin_bfgs

def l1_loss(x, info):
    res = (np.abs(x-ranks[:,np.newaxis])**1).sum()
    
    if info['Nfeval']%1==0:
        print(info['Nfeval'], x)
    info['Nfeval']+=1
    return res


fmin_bfgs(l1_loss, 
             x0=[0], 
             callback=callback,
            args=({'Nfeval':0},),
          full_output=True
            )

In [ ]:
#import autograd
import autograd.numpy as np

In [ ]:
l1_loss = lambda x : np.abs(x-ranks[:,np.newaxis]).sum()
learned_median = minimize(l1_loss, x0=0)
print(learned_median['x'][0], np.median(ranks))

In [ ]:
np.isclose(learned_median['x'][0], np.median(ranks), rtol=1e-3)

In [ ]:
import autograd.numpy as np  # Thinly-wrapped numpy
from autograd import grad


def l1_loss(x):
    return np.abs(x-ranks[:,np.newaxis]).sum()

def l2_loss(x):
    return (np.abs(x-ranks[:,np.newaxis])**2).sum()


l1_parameter = 0.0
l1_progress = list()
learning_rate = 1e-4

for _ in tqdm_notebook(range(500)):
    g = grad(l1_loss)(l1_parameter)
    l1_parameter = l1_parameter + -1*g*learning_rate
    l1_progress.append(l1_parameter)
    
l2_parameter = 0.0
l2_progress = list()
learning_rate = 1e-5
#
for _ in tqdm_notebook(range(500)):
    g = grad(l2_loss)(l2_parameter)
    l2_parameter = l2_parameter + -1*g*learning_rate
    l2_progress.append(l2_parameter)

In [ ]:
plt.plot(l1_progress, label='l1 loss')
plt.plot(l2_progress, label='l2 loss')
plt.axhline(np.mean(ranks), label='True mean', linestyle='--', c='k')

plt.axhline(np.median(ranks), label='True median', c='k', linestyle='--')

plt.legend()

In [ ]:
plt.plot(x,bapsum)
print(x[minimum_index])
plt.axvline(x[minimum_index])


In [ ]:
import pymc3 as pm

with pm.Model() as model:
    #prior:
    m = pm.Normal('m', mu=5, sigma=6.0)
    bee = pm.HalfNormal('bee', sigma=6.0)
    #likelihood:
    y = pm.Laplace('y', mu=m, b=bee,observed=ranks)

In [ ]:
with model:
    trace = pm.sample(draws=500, tune=500, chains=2,
                      target_accept=0.9)


In [ ]:
kdeplot(trace['m'])
plt.vlines(pm.stats.hpd(trace['m']),0,0.6)
plt.axvline(x[minimum_index])

import scikits.bootstrap as boot
boot.ci(ranks, np.median, n_samples=1000)


In [ ]:
from scipy.stats import betaprime


ranks = betaprime(a=100, b=3, loc=0.0, scale=10.0).rvs(2500)

plt.hist(ranks, alpha=0.3, bins=100)
print(ranks.mean(), np.median(ranks))
plt.axvline(np.mean(ranks), label='mean')

plt.axvline(np.median(ranks), label='median')

In [ ]:

from scipy.stats import wald


ranks = wald(loc=0.0,scale=30).rvs(2500)

plt.hist(ranks, alpha=0.3, bins=100)
print(ranks.mean(), np.median(ranks))
plt.axvline(np.mean(ranks), label='mean')

plt.axvline(np.median(ranks), label='median')